# 第8章 分类数据

In [1]:
import pandas as pd
import numpy as np


## 四、问题与练习

#### 【问题一】 如何使用union_categoricals方法？它的作用是什么？
分类数据经过 pd.concat 合并后类型转为了 object 类型，如果想要保持分类类型的话，可以借助 union_categoricals 来完成。  
`from pandas.api.types import union_categoricals 
union_categoricals([blood_type1, blood_type2])`


In [162]:
foo = pd.Categorical(['a', 'b', 'a', 'd'], categories=['a', 'b', 'c', 'd'])
bar = pd.Categorical(['d', 'e', 'd', 'e'], categories=['d', 'e', 'f', 'g'])
from pandas.api.types import union_categoricals 
display(union_categoricals([pd.Series(foo), pd.Series(bar)]),pd.concat([pd.Series(foo),pd.Series(bar)]))

[a, b, a, d, d, e, d, e]
Categories (7, object): [a, b, c, d, e, f, g]

0    a
1    b
2    a
3    d
0    d
1    e
2    d
3    e
dtype: object

#### 【问题二】 利用concat方法将两个序列纵向拼接，它的结果一定是分类变量吗？什么情况下不是？
不是，见问题一，使用union_categoricals时结果是分类变量，pd.concat 合并后类型转为了 object 类型

#### 【问题三】 当使用groupby方法或者value_counts方法时，分类变量的统计结果和普通变量有什么区别？

In [166]:
#此处使用了练习一的数据。未发现明显区别
display(df1['deep_cat'],  df1['deep_cat'].value_counts(),  df1['方向'].value_counts())

0         Ⅲ
1         Ⅱ
2         Ⅰ
3         Ⅲ
4         Ⅱ
         ..
10057     Ⅱ
10058     Ⅰ
10059     Ⅱ
10060    VI
10061     Ⅱ
Name: deep_cat, Length: 10062, dtype: category
Categories (7, object): [Ⅰ < Ⅱ < Ⅲ < IV < Ⅴ < VI < VII]

Ⅱ      3744
Ⅲ      2349
Ⅰ      1816
VI      763
Ⅴ       530
IV      501
VII     359
Name: deep_cat, dtype: int64

north_west    2019
south_west    2011
south_east    1917
north_east    1901
south          605
north          576
east           536
west           497
Name: 方向, dtype: int64

In [169]:
display(df1.groupby('方向').mean(),df1.groupby('deep_cat').mean())

,维度,经度,距离,深度,烈度
方向,,,,,
east,38.846119,33.400914,2.637687,12.417351,1.620336
north,39.032813,33.131510,4.510069,11.563368,1.683333
north_east,38.879658,33.845592,3.175539,12.787533,1.590742
north_west,38.864814,33.447098,3.481625,12.399950,1.699752
south,38.760463,33.899521,2.851240,12.096860,1.690083
south_east,38.716166,33.789280,3.244810,12.963850,1.654147
south_west,38.726718,33.437295,2.810094,12.438140,1.756937
west,38.771187,34.077223,2.561167,11.974849,1.747686


,维度,经度,距离,深度,烈度
deep_cat,,,,,
Ⅰ,38.800347,33.610644,2.865694,1.725496,1.047797
Ⅱ,38.882009,34.426012,3.118643,6.244177,1.827431
Ⅲ,38.880430,32.389434,3.231205,10.760196,1.323329
IV,38.656667,31.980439,3.428343,16.572455,1.788822
Ⅴ,38.546849,33.470340,3.414151,23.927170,1.863962
VI,38.770367,34.324666,3.506160,35.747313,2.799214
VII,38.317493,34.420446,3.549582,71.723955,2.830362


#### 【问题四】 下面的代码说明了Series创建分类变量的什么“缺陷”？如何避免？（提示：使用Series中的copy参数）

In [141]:
#”缺陷“：创建时指向了同一块内存，修改s时cat也同步变化

cat = pd.Categorical([1, 2, 3, 10], categories=[1, 2, 3, 4, 10])
s = pd.Series(cat, copy=True, name="cat")#方法一：使用copy参数
#s = pd.Series(cat.copy(), name="cat")   #方法二：.copy()
display(cat)
s.iloc[0:2] = 10
display(s,cat)

[1, 2, 3, 10]
Categories (5, int64): [1, 2, 3, 4, 10]

0    10
1    10
2     3
3    10
Name: cat, dtype: category
Categories (5, int64): [1, 2, 3, 4, 10]

[1, 2, 3, 10]
Categories (5, int64): [1, 2, 3, 4, 10]

#### 【练习一】 现继续使用第四章中的地震数据集，请解决以下问题：
#### （a）现在将深度分为七个等级：[0,5,10,15,20,30,50,np.inf]，请以深度等级Ⅰ,Ⅱ,Ⅲ,Ⅳ,Ⅴ,Ⅵ,Ⅶ为索引并按照由浅到深的顺序进行排序。
#### （b）在（a）的基础上，将烈度分为4个等级：[0,3,4,5,np.inf]，依次对南部地区的深度和烈度等级建立多级索引排序。

In [61]:
#(a)现在将深度分为七个等级：[0,5,10,15,20,30,50,np.inf]，请以深度等级Ⅰ,Ⅱ,Ⅲ,Ⅳ,Ⅴ,Ⅵ,Ⅶ为索引并按照由浅到深的顺序进行排序。
df1 = pd.read_csv('data/Earthquake.csv')
df1.head()
df1['deep_cat']=pd.cut(df1['深度'], [0,5,10,15,20,30,50,np.inf],right=False, labels=['Ⅰ','Ⅱ','Ⅲ','IV','Ⅴ','VI','VII'])
df1.set_index('deep_cat',drop=False).sort_index()

,日期,时间,维度,经度,方向,距离,深度,烈度,deep_cat
deep_cat,,,,,,,,,
Ⅰ,1999.07.05,12:57:31 AM,41.24,32.78,north_west,0.9,0.0,0.0,Ⅰ
Ⅰ,1994.12.24,12:40:49 AM,37.48,28.35,north_east,1.0,0.0,0.0,Ⅰ
Ⅰ,2001.12.12,12:59:55 AM,37.42,37.28,north_east,3.2,4.0,0.0,Ⅰ
Ⅰ,2017.04.21,12:25:56 AM,38.78,29.06,south_east,1.5,3.4,3.8,Ⅰ
Ⅰ,1992.08.21,12:10:52 AM,37.61,27.48,north_east,1.0,0.0,0.0,Ⅰ
...,...,...,...,...,...,...,...,...,...
VII,1981.08.04,12:52:19 AM,38.90,37.00,north,3.7,84.0,0.0,VII
VII,1968.03.21,12:42:51 AM,38.80,27.60,south,0.6,52.0,4.3,VII
VII,1929.04.27,12:18:06 AM,40.51,31.43,south_west,1.9,70.0,4.8,VII


In [66]:
#(b)在（a）的基础上，将烈度分为4个等级：[0,3,4,5,np.inf]，依次对南部地区的深度和烈度等级建立多级索引排序。
df1['intensity_cat'] = pd.cut(df1['烈度'],[0,3,4,5,np.inf],right=False, labels=['I','Ⅱ','Ⅲ','IV'])
df1[df1['方向']=='south'].set_index(['deep_cat','intensity_cat'],drop=False).sort_index()

日期           时间     维度     经度     方向   距离  \
deep_cat intensity_cat                                                      
Ⅰ        I              2000.05.14  12:18:59 AM  38.60  39.92  south  0.3   
         I              1982.01.05  12:53:17 AM  39.61  28.50  south  0.5   
         I              1986.07.20  12:01:18 AM  37.81  35.91  south  0.5   
         I              1996.08.14  12:14:37 AM  40.70  35.43  south  0.5   
         I              2002.09.16  12:54:15 AM  41.19  34.16  south  0.5   
...                            ...          ...    ...    ...    ...  ...   
VII      Ⅲ              1933.01.02  12:56:58 AM  38.01  38.24  south  2.6   
         Ⅲ              1936.08.02  12:21:09 AM  37.88  29.70  south  2.9   
         Ⅲ              1971.04.30  12:10:04 AM  37.76  36.18  south  3.5   
         Ⅲ              1964.11.20  12:59:19 AM  40.20  28.06  south  3.6   
         Ⅲ              1970.03.30  12:15:44 AM  38.96  29.73  south  4.0   

                          深度   烈度 deep_cat intensity_cat  
deep_cat intensity_cat                                    
Ⅰ        I               0.0  0.0        Ⅰ             I  
         I               2.0  0.0        Ⅰ             I  
         I               2.0  0.0        Ⅰ             I  
         I               0.0  0.0        Ⅰ             I  
         I               3.0  0.0        Ⅰ             I  
...                      ...  ...      ...           ...  
VII      Ⅲ              50.0  4.6      VII             Ⅲ  
         Ⅲ              70.0  4.8      VII             Ⅲ  
         Ⅲ              60.0  4.3      VII             Ⅲ  
         Ⅲ              56.0  4.2      VII             Ⅲ  
         Ⅲ              55.0  4.3      VII             Ⅲ  

[605 rows x 10 columns]

#### 【练习二】 对于分类变量而言，调用第4章中的变形函数会出现一个BUG（目前的版本下还未修复）：例如对于crosstab函数，按照[官方文档的说法](https://pandas.pydata.org/pandas-docs/version/1.0.0/user_guide/reshaping.html#cross-tabulations)，即使没有出现的变量也会在变形后的汇总结果中出现，但事实上并不是这样，比如下面的例子就缺少了原本应该出现的行'c'和列'f'。基于这一问题，请尝试设计my_crosstab函数，在功能上能够返回正确的结果。

In [114]:
foo = pd.Categorical(['a', 'b', 'a','c'], categories=['a', 'b', 'c','d'])
bar = pd.Categorical(['d', 'e','d', 'e'], categories=['d', 'e', 'f'])
pd.crosstab(foo, bar)


col_0,d,e
row_0,,
a,2,0
b,0,1
c,0,1


In [115]:
def my_crosstab(foo,bar):
    crosstab = pd.crosstab(foo, bar)
    #找到未出现项，在crosstab基础上补0
    for i in foo.categories:
        if i in foo.remove_unused_categories().categories:pass
        else:crosstab.loc[i]=0
    for j in bar.categories:
        if j in bar.remove_unused_categories().categories:pass
        else:crosstab[j]=0
    return crosstab
my_crosstab(foo,bar)

col_0,d,e,f
row_0,,,
a,2,0,0
b,0,1,0
c,0,1,0
d,0,0,0
